--- Day 7: Recursive Circus ---  
Wandering further through the circuits of the computer, you come upon a tower of programs  
that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand,  
and now they're balanced precariously in a large tower.  

One program at the bottom supports the entire tower.  
It's holding a large disc, and on the disc are balanced several more sub-towers.  
At the bottom of these sub-towers, standing on the bottom disc, are other programs,  
each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers,  
many programs stand simply keeping the disc below them balanced but with no disc of their own.  

You offer to help, but first you need to understand the structure of these towers.  
You ask each program to yell out their name, their weight, and (if they're holding a disc)  
the names of the programs immediately above them balancing on that disc. You write this information down  
(your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion;  
by the time you're done, you're not sure which program gave which information.  

For example, if your list is the following:  

pbga (66)  
xhth (57)  
ebii (61)  
havc (66)  
ktlj (57)  
fwft (72) -> ktlj, cntj, xhth  
qoyq (66)  
padx (45) -> pbga, havc, qoyq  
tknk (41) -> ugml, padx, fwft  
jptl (61)  
ugml (68) -> gyxo, ebii, jptl  
gyxo (61)  
cntj (57)  
...then you would be able to recreate the structure of the towers that looks like this:  

                gyxo  
              /      
         ugml - ebii  
       /      \       
      |         jptl  
      |          
      |         pbga    
     /        /  
tknk --- padx - havc  
     \        \  
      |         qoyq  
      |               
      |         ktlj  
       \      /       
         fwft - cntj  
              \       
                xhth  
In this example, tknk is at the bottom of the tower (the bottom program), and is holding up ugml, padx, and fwft.   Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any   other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much   larger.)  

Before you're ready to help them, you need to make sure your information is correct.  
What is the name of the bottom program?  

In [5]:
import numpy as np
import pandas as pd

In [6]:
def parse(string):
    """Parse out name, weight and children from input string"""
    name=''
    weight=0
    childs=[]
    
    if '->' in string:
        #padx (45) -> pbga, havc, qoyq is <name>(<weight>) -> <childs...>
        front,back1=string.split(' (')
        name=str(front)
        
        num,back2=back1.split(')')
        weight=int(num)
        
        back3=back2.replace(' -> ','')
        back4=back3.replace(',','')
        back5=back4.split(' ')
        childs=back5
        
    else:
        #pbga (66) is <name>(<weight>)
        front,back=string.split(' (')
        name=str(front)
        weight=int(back[:-1])
    
    return name,weight,childs


In [7]:
parse('pbga (66)')

('pbga', 66, [])

In [8]:
parse('fwft (72) -> ktlj, cntj, xhth')

('fwft', 72, ['ktlj', 'cntj', 'xhth'])

In [9]:
test=[
'pbga (66)',
'xhth (57)', 
'ebii (61)', 
'havc (66)',
'ktlj (57)', 
'fwft (72) -> ktlj, cntj, xhth', 
'qoyq (66)', 
'padx (45) -> pbga, havc, qoyq', 
'tknk (41) -> ugml, padx, fwft',  
'jptl (61)',  
'ugml (68) -> gyxo, ebii, jptl', 
'gyxo (61)', 
'cntj (57)'
]

In [10]:
test_parsed=[parse(t) for t in test]

In [11]:
test_parsed

[('pbga', 66, []),
 ('xhth', 57, []),
 ('ebii', 61, []),
 ('havc', 66, []),
 ('ktlj', 57, []),
 ('fwft', 72, ['ktlj', 'cntj', 'xhth']),
 ('qoyq', 66, []),
 ('padx', 45, ['pbga', 'havc', 'qoyq']),
 ('tknk', 41, ['ugml', 'padx', 'fwft']),
 ('jptl', 61, []),
 ('ugml', 68, ['gyxo', 'ebii', 'jptl']),
 ('gyxo', 61, []),
 ('cntj', 57, [])]

In [12]:
def find_bottom_name(array_of_parse):
    """Find the bottom name that is supporting other names, but noone supports him"""
    all_names=[]
    all_childs=[]
    
    for i in array_of_parse:
        name=i[0]
        childs=i[2]
        
        if childs!=[]:
            all_names.append(name)
            all_childs.extend(childs)
    
    for n in all_names:
        if n not in all_childs:
            return n
    
    return 'No bottom found'

In [13]:
find_bottom_name(test_parsed)

'tknk'

In [14]:
with open('inputs.txt','r') as f:
    inputs=f.readlines()
    inputs=[i.replace('\n','') for i in inputs]

In [15]:
inputs[:5]

['hndzyx (28)',
 'kdelzk (399)',
 'uylvg (403) -> xrvcjq, hihltxf, junpjcj, onkvtu, cckdoyb, favay, xsygurk',
 'jiyqd (842) -> vunjzqr, sfooljn, gsnlv',
 'mxmzyac (67) -> pbwmrq, eiqjanp, dyanei, zmkuj, vkvvtm, ktlhztg']

In [16]:
find_bottom_name([parse(i) for i in inputs])

'hlqnsbe'

--- Part Two ---  
The programs explain the situation: they can't get down. Rather, they could get down,  
if they weren't expending all of their energy trying to keep the tower balanced.  
Apparently, one program has the wrong weight, and until it's fixed, they're stuck here.

For any program holding a disc, each program standing on that disc forms a sub-tower.  
Each of those sub-towers are supposed to be the same weight,  
or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.  

In the example above, this means that for ugml's disc to be balanced,  
gyxo, ebii, and jptl must all have the same weight, and they do: 61.  

However, for tknk to be balanced, each of the programs standing on its disc  
and all programs above it must each match. This means that the following sums must all be the same:  

ugml + (gyxo + ebii + jptl) = 68 + (61 + 61 + 61) = 251  
padx + (pbga + havc + qoyq) = 45 + (66 + 66 + 66) = 243  
fwft + (ktlj + cntj + xhth) = 72 + (57 + 57 + 57) = 243  
As you can see, tknk's disc is unbalanced: ugml's stack is heavier than the other two.  
Even though the nodes above ugml are balanced, ugml itself is too heavy:  
it needs to be 8 units lighter for its stack to weigh 243 and keep the towers balanced.  
If this change were made, its weight would be 60.

Given that exactly one program is the wrong weight,  
what would its weight need to be to balance the entire tower?  

In [18]:
def get_info(name,array_of_parsed):
    """Input:name, Output:name,weight,child"""
    for i in array_of_parsed:
        if i[0]==name:
            return i[0],i[1],i[2]
    
    return 'Failed',0,['Failed']

In [23]:
def get_weight(name,array_of_parsed):
    """Calculate weight supported at node 'name'"""
    n1,weight,childs=get_info(name,array_of_parsed)
    
    if childs==[]:
        return weight
    else:
        for i in childs:
            n,w,c=get_info(i,array_of_parsed)
            weight+=get_weight(n,array_of_parsed)
    return weight
                

In [24]:
get_info('ugml',test_parsed)

('ugml', 68, ['gyxo', 'ebii', 'jptl'])

In [25]:
get_weight('ugml',test_parsed)

251

In [34]:
### stuck here

def check_weights(name,array_of_parsed):
    """Go down the tree, check if childs have all same weight"""
    n1,weight,childs=get_info(name,array_of_parsed)
    
    child_names=[]
    child_weights=[]
    if childs!=[]:
        for c in childs:
            child_names.append(c)
            child_weights.append(get_weight(c,array_of_parsed))
    
    if all(x==child_weights[0] for x in child_weights)==False: #Check if all terms are the same in list
        return child_names,child_weights

In [36]:
for t in test_parsed:
    check_weights(t[0],test_parsed)

In [37]:
check_weights('ugml',test_parsed)